# Trading Decision Simulator

## Description
This document provides Python functions to simulate trading decisions using a predefined API. The API includes stock tickers, historical prices, and a `Trade` class to represent buy or sell actions. Each function demonstrates a unique trading strategy, such as momentum-based trading, mean reversion, portfolio diversification, and more. These examples can serve as a foundation for developing or testing algorithmic trading systems.


In [0]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from huggingface_hub import login, InferenceClient
from transformers import AutoTokenizer
import google.generativeai as google_genai
import anthropic
import gradio as gr

In [0]:
# Setting up environment
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')
os.environ['CODE_QWEN_URL'] = os.getenv('CODE_QWEN_URL', 'your-url-if-not-using-env')

In [0]:
# Initialize
openai = OpenAI()
claude = anthropic.Anthropic()
google_genai.configure()
code_qwen = InferenceClient(CODE_QWEN_URL)

In [0]:
# Constants 
MODELS = {
    "GPT": "gpt-4o", 
    "Claude": "claude-3-5-sonnet-20240620", 
    "Gemini": "gemini-1.5-pro", 
    "CodeQwen": "Qwen/CodeQwen1.5-7B-Chat"
}

MAX_TOKENS = 2000

SYSTEM_PROMPT = """
You are an advanced code generation assistant capable of creating high-quality Python code for financial trading systems. 
Your task is to generate Python functions that simulate trading decisions based on the following API:

API DETAILS:
1. tickers: A list of stock tickers (strings) representing available stocks.
2. prices: A dictionary where the key is a stock ticker (string) and the value is a list of historical prices (floats). The list is ordered with the most recent price first.
3. Trade: A class used to represent trading actions.
   - `Trade(ticker, quantity)` creates a trade object:
     - Positive `quantity` (e.g., `100`) represents buying shares.
     - Negative `quantity` (e.g., `-50`) represents selling/shorting shares.

INSTRUCTIONS:
- You will be provided with an example Python function to demonstrate the API.
- Your job is to generate 5 additional Python functions, each implementing a unique trading strategy.
- Ensure the functions are named sequentially (e.g., `trade2()`, `trade3()`, etc.).
- Include clear comments explaining the logic behind each function.
- Return a list of `Trade` objects from each function.
- The output should only include the Python code. Do not include any introductions, conclusions, summaries, or additional context.

CONSIDERATIONS FOR TRADING STRATEGIES:
- Momentum-based strategies (e.g., trading based on price trends).
- Mean reversion strategies (e.g., identifying overbought or oversold stocks).
- Randomized strategies (e.g., simulating stochastic decision-making).
- Portfolio diversification (e.g., distributing trades across multiple tickers).
- Risk management strategies (e.g., limiting losses or locking in profits).

EXAMPLE FUNCTION:
"""

In [0]:
def user_prompt(example_function):
    """
    Returns a user prompt for the model by appending the provided example function.
    """
    return f"""
{example_function}

TASK:
Based on the provided example function and API, please write 5 additional trading functions named `trade2()`, `trade3()`, and so on. Each function should implement a unique trading strategy as outlined in the system prompt. Make sure each function has clear comments explaining the logic and returns a list of `Trade` objects.
"""

# Trade Function Example: `trade1`

This Python script demonstrates a simple trading strategy implemented using a provided API. The `trade1` function identifies the top-performing stock over the last 5 days based on its average price and creates a trade object to buy 100 shares of the selected stock. The function leverages the following components:

- **`tickers`**: A list of available stock tickers.
- **`prices`**: A dictionary containing historical prices for each stock.
- **`Trade`**: A class used to represent trading actions (buy or sell).
- **`numpy`**: Used to calculate average prices efficiently.

The example highlights a momentum-based strategy where the stock with the best recent performance is selected for trading.

example:
```python
# Importing the required modules and classes for the trading simulation

# `tickers` is a list of stock tickers (strings), representing available stocks to trade.
import tickers

# `prices` is a dictionary where:
#   - The key is a stock ticker (string).
#   - The value is a list of historical prices (floats), ordered with the most recent price first.
import prices

# `Trade` is a class that represents a trading decision. It takes two arguments:
#   - `ticker`: A string representing the stock ticker.
#   - `quantity`: An integer representing the number of shares to buy (positive) or sell/short (negative).
# Example usage:
#   Trade("IBM", 100)  -> Buys 100 shares of IBM stock.
#   Trade("IBM", -50)  -> Sells or shorts 50 shares of IBM stock.
import Trade

# Additional modules for random number generation and numerical operations
import random
import numpy as np

def trade1():
    """
    Buys the top-performing stock based on its average price over the last 5 days.

    Strategy:
    - Calculate the average price of the last 5 days for each stock in `tickers`.
    - Identify the stock with the highest average price.
    - Create a trade object to buy 100 shares of the identified stock.
    
    Returns:
        list[Trade]: A list containing a single trade object for the chosen stock.
    """
    # Calculate the 5-day average price for each stock
    avg_prices = {ticker: np.mean(prices[ticker][:5]) for ticker in tickers}

    # Find the stock ticker with the highest 5-day average price
    best_ticker = max(avg_prices, key=avg_prices.get)

    # Create a trade object to buy 100 shares of the top-performing stock
    trade = Trade(best_ticker, 100)

    # Return the trade as a list
    return [trade]

```

In [0]:
# A trading function example
TRADING_FUNCTION_EXAMPLE = """
# tickers is a list of stock tickers (strings)
import tickers

# prices is a dict; the key is a ticker and the value is a list of historic prices, today first
import prices

# Trade represents a decision to buy or sell a quantity of a ticker
# Trade("IBM", 100) for a trade object representing purchasing 100 shares of IBM stock
# Trade("IBM", -50) for a trade object representing selling or shorting 50 shares of IBM stock
import Trade

import random
import numpy as np

def trade1():
    # Buy top performing stock in the last 5 days
    avg_prices = {ticker: np.mean(prices[ticker][:5]) for ticker in tickers}
    best_ticker = max(avg_prices, key=avg_prices.get)
    trade = Trade(best_ticker, 100)
    return [trade]
"""

In [0]:
# UI function to trade using GPT
def trade_gpt(function_example):    
    stream = openai.chat.completions.create(
        model=MODELS["GPT"], 
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt(function_example)}
        ], 
        stream=True
    )
    reply = ""
    for chunk in stream:
        reply += chunk.choices[0].delta.content or ""
        yield reply.replace("```python\n", "").replace("```", "")

In [0]:
# UI function to trade using Claude
def trade_claude(function_example):
    result = claude.messages.stream(
        model=MODELS["Claude"],
        max_tokens=MAX_TOKENS,
        system=SYSTEM_PROMPT,
        messages=[{"role": "user", "content": user_prompt(function_example)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace("```python\n", "").replace("```", "")

In [0]:
# UI function to trade using Gemini
def trade_gemini(function_example):
    gemini = google_genai.GenerativeModel(
        model_name=MODELS["Gemini"],
        system_instruction=SYSTEM_PROMPT
    )
    stream = gemini.generate_content(
        contents=user_prompt(function_example),
        stream=True
    )
    reply = ""
    for chunk in stream:
        reply += chunk.text or ""
        yield reply.replace("```python\n", "").replace("```", "")

In [0]:
# UI function to trade using CodeQwen
def trade_code_qwen(function_example):
    tokenizer = AutoTokenizer.from_pretrained(MODELS["CodeQwen"])
    model_input = tokenizer.apply_chat_template(
        conversation=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt(function_example)}
        ],
        tokenize=False,
        add_generation_prompt=True
    )
    stream = code_qwen.text_generation(
        prompt=model_input,
        stream=True,
        details=True,
        max_new_tokens=MAX_TOKENS
    )
    reply = ""
    for chunk in stream:
        reply += chunk.token.text or ""
        yield reply.replace("```python\n", "").replace("```", "")

In [0]:
# UI function to select model from dropdown 
def trade(trading_function, model):
    if model=="GPT":
        yield from trade_gpt(trading_function)
    elif model=="Claude":
        yield from trade_claude(trading_function)
    elif model=="Gemini":
        yield from trade_gemini(trading_function)
    elif model=="CodeQwen":
        yield from trade_code_qwen(trading_function)
    else:
        raise ValueError("Unknown Model")

In [0]:
# CSS styling for the UI
UI_CSS = """
#title {
    text-align: center;
    font-size: 2.5em;
    font-weight: bold;
    margin-bottom: 10px;
}

#description {
    text-align: left;
    font-size: 1.2em;
    font-weight: bold;
    margin-bottom: 20px;
    color: #BBB;
}

#simulate-btn {
    height: 3em;
    font-size: 2em !important;
    padding: 12px 25px !important;
    border-radius: 10px !important;
    border: none !important;
    cursor: pointer;
    transition: background-color 0.3s, transform 0.2s; /* Smooth effects */
}

#simulate-btn:hover {
    background-color: #FFC107 !important; /* Bright golden-yellow on hover */
    transform: scale(1.05); /* Slight zoom effect */
    box-shadow: 0 6px 8px rgba(0, 0, 0, 0.25); /* Enhance shadow on hover */
}

#simulate-btn:active {
    background-color: #B8860B !important; /* Darker goldenrod on click */
    transform: scale(0.95); /* Slight press effect */
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.2); /* Reduce shadow on click */
}

#simulate-btn,
#trading-decisions {
    background-color: #DAA520 !important; /* Goldenrod color same as #simulate-btn */
    color: #FFFFFF !important; /* White text for contrast */
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.3); /* Subtle shadow for depth */
}

#trading-decisions {
    border: 2px solid #B8860B; /* Darker goldenrod border */
}

#trading-decisions:focus {
    outline: none;
    box-shadow: 0 0 8px #FFC107; /* Bright golden-yellow glow on focus */
}

#example-function, 
#model-dropdown {
    background-color: #007965 !important; /* Darker and sharper teal for better contrast */
    color: #FFFFFF !important; /* Pure white for strong visibility */
    cursor: pointer;
    border: 2px solid #00594D; /* Deep teal border for emphasis */
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.9); /* Strong shadow for depth */
}

#example-function:focus,
#model-dropdown:focus {
    outline: none;
    box-shadow: 0 0 10px #00A389; /* Vibrant teal glow on focus */
}

#model-dropdown:hover {
    background-color: #005F4A !important; /* Darker teal for hover effect */
    box-shadow: 0 6px 10px rgba(0, 0, 0, 0.4); /* Enhanced shadow on hover */
    border-color: #00A389; /* Change border color for hover */
}
"""

In [0]:
# Gradio UI
with gr.Blocks(css=UI_CSS) as ui:
    # Title for the application
    gr.Markdown("<div id='title'>🛠️ Trading Strategy Simulator</div><br>")
    
    # Input and output section
    with gr.Row():
        trading_f = gr.Textbox(
            label="📄 Trading Function Input",
            placeholder="Paste your trading function here...",
            lines=15,
            value=TRADING_FUNCTION_EXAMPLE,
            elem_id="example-function"
        )
        decisions = gr.Textbox(
            label="📊 Generated Trading Decisions",
            placeholder="Trading decisions will appear here...",
            lines=20,
            interactive=False,
            elem_id="trading-decisions"
        )
    
    with gr.Row():
        # Dropdown scaled to take 1 part of the row
        model = gr.Dropdown(
            choices=MODELS,
            label="🤖 Select AI Model",
            value="GPT",
            scale=1,
            elem_id="model-dropdown"
        )
        # Markdown for the description scaled to 2 parts of the row
        with gr.Column(scale=2):
            gr.Markdown(
                """
                <div id='description'>
                This interface allows you to test and simulate trading strategies using a predefined example function.
                Simply input a trading function, select your preferred AI model, and see the generated trading decisions in action.<br>
                Experiment with different strategies to refine your approach and analyze outcomes effectively.
                </div>
                """
            )
        # Button scaled to take 1 part of the row
        trade_btn = gr.Button(
            "💼 Simulate Trading",
            elem_id="simulate-btn",
            scale=1
        )

    # Action button behavior
    trade_btn.click(
        fn=trade, 
        inputs=[trading_f, model], 
        outputs=[decisions]
    )

# Launch the UI in a browser
ui.launch(inbrowser=True)